In [40]:

import skimage as ski
import matplotlib.pyplot as plt
import math
import ipywidgets
import numpy as np
import pydicom
from pydicom import uid
from pydicom.dataset import Dataset, FileDataset
import datetime
import random


from IPython.display import display



In [41]:
def load_img(filename):
    image = ski.io.imread(filename, as_gray=True)
    image = ski.img_as_float(image)
    return image

In [42]:
#globals
reconstructed_image = None

In [43]:
def normalize(image):
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    image = image * 255
    return image

In [44]:
def bresenham(x1,y1, x2, y2):
    points = []
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy

    while True:
        points.append((x1, y1))
        if x1 == x2 and y1 == y2:
            break
        err2 = err * 2
        if err2 > -dy:
            err -= dy
            x1 += sx
        if err2 < dx:
            err += dx
            y1 += sy

    return points


In [45]:
def radon_transform(image, angle=360, detectors=180, scans=360, angle_of_cone=90):  #scans = steps, angle = rotation angle
    sinogram_steps = []

    rows, cols = image.shape
    center_y = rows // 2
    center_x = cols // 2

    emiters_positions = []

    radius = np.sqrt(rows**2 + cols**2) / 2
    sinogram = np.zeros((scans, detectors), dtype=image.dtype)
    delta_alpha = angle / scans
    for iteration in range(scans):
        alpha = iteration * delta_alpha
        x_e = int(center_x + radius * math.cos(math.radians(alpha)))
        y_e = int(center_y + radius * math.sin(math.radians(alpha)))
        emiters_positions.append((x_e, y_e))

        for i in range(detectors):  # i as a detector number
            x_di = int(center_x + radius * math.cos(math.radians(alpha + 180 - angle_of_cone + i * 2 * angle_of_cone / (detectors - 1))))
            y_di = int(center_y + radius * math.sin(math.radians(alpha + 180 - angle_of_cone + i * 2 * angle_of_cone / (detectors - 1))))

            points = bresenham(x_e, y_e, x_di, y_di)

            total_intensity = 0
            valid_points_count = 0
            for x_b, y_b in points:
                if 0 <= y_b < rows and 0 <= x_b < cols:
                    total_intensity += image[y_b, x_b]
                    valid_points_count += 1

            if valid_points_count > 0:
                total_intensity /= valid_points_count
            sinogram[iteration, i] = total_intensity
        sinogram_steps.append(sinogram.copy())

    return sinogram, sinogram_steps, emiters_positions


In [46]:
def inverse_radon_transform(sinogram, image_shape, angle=360, angle_of_cone=90):
    steps = []
    scans, detectors = sinogram.shape
    rows, cols = image_shape
    center_y = rows // 2
    center_x = cols // 2
    radius = np.sqrt(rows**2 + cols**2) / 2
    delta_alpha = angle / scans
    reconstruction = np.zeros((rows, cols), dtype=np.float32)

    for iteration in range(scans):
        alpha = iteration * delta_alpha
        angle_rad = math.radians(alpha)
        x_e = int(center_x + radius * math.cos(angle_rad))
        y_e = int(center_y + radius * math.sin(angle_rad))

        for i in range(detectors):
            detector_angle = math.radians(alpha + 180 - angle_of_cone + i * 2 * angle_of_cone / (detectors - 1))
            x_d = int(center_x + radius * math.cos(detector_angle))
            y_d = int(center_y + radius * math.sin(detector_angle))


            points = bresenham(x_e, y_e, x_d, y_d)

            value = sinogram[iteration, i]

            for x_b, y_b in points:
                if 0 <= y_b < rows and 0 <= x_b < cols:
                    reconstruction[y_b, x_b] += value
        steps.append(reconstruction.copy())



    return reconstruction, steps

In [47]:

def save_dicom(b, output):
    with output:
        print("🔔 Save button clicked!")
    global reconstructed_image
    if reconstructed_image is None:
        with output:
            print("No image to save. Please run the reconstruction first.")
    else:
        with output:
            print("There is an image to save")
    filename = output_file_name.value + ".dcm"
    ds = pydicom.Dataset()


    file_meta = pydicom.dataset.FileMetaDataset()
    file_meta.MediaStorageSOPClassUID = pydicom.uid.SecondaryCaptureImageStorage
    file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    file_meta.ImplementationClassUID = pydicom.uid.generate_uid()
    file_meta.ImplementationVersionName = "PYDICOM 2.0.0"

    ds.file_meta = file_meta

    ds.PatientName = patient_name.value
    ds.PatientID = patient_id.value
    ds.PatientBirthDate = patient_birth.value
    ds.PatientSex = patient_sex.value
    ds.ExamDate = date_of_exam.value

    ds.Modality = "CT"
    ds.StudyDate = date_of_exam.value

    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.SOPInstanceUID = pydicom.uid.generate_uid()
    ds.SOPClassUID = pydicom.uid.CTImageStorage
    ds.ImageType = ["ORIGINAL", "PRIMARY", "AXIAL"]
    ds.InstanceNumber = str(random.randint(1, 100))
    ds.ImagesInAcquisition = "1"
    ds.FrameOfReferenceUID = pydicom.uid.generate_uid()
    ds.ImageComments = comment.value

    ds.Rows, ds.Columns = reconstructed_image.shape
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.SamplesPerPixel = 1
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15
    ds.PixelRepresentation = 0
    ds.PixelData = (reconstructed_image.astype(np.uint16)).tobytes()

    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.save_as(filename, write_like_original=False)
    with output:
        print("DICOM file saved successfully.")


In [48]:
def show_steps(n, steps, name):
    plt.imshow(steps[n-1], cmap='gray')
    plt.title(name + " : Step " + str(n))
    plt.axis("off")
    plt.show()


In [49]:
def interactive_main(filename, steps, detectors, angle_of_cone):
    global reconstructed_image
    image = load_img(filename)
    sinogram, steps_sinogram, emiters = radon_transform(image, steps, detectors, steps, angle_of_cone)
    reversed, steps_inversed = inverse_radon_transform(sinogram, image.shape, steps, angle_of_cone)

    reversed = normalize(reversed.copy())
    reconstructed_image = reversed.copy()
    steps_inversed = [normalize(step.copy()) for step in steps_inversed]

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(image, cmap='gray')
    axes[0].set_title("Obraz wejściowy")
    axes[0].axis("off")
    # if emiters:
    #     xs, ys = zip(*emiters)
    #     axes[0].scatter(xs, ys, c='red', s=10, label='Emiters')

    axes[1].imshow(sinogram, cmap='gray')
    axes[1].set_title("Sinogram")
    axes[1].axis("off")

    axes[2].imshow(reversed, cmap='gray')
    axes[2].set_title("Obraz wyjściowy")
    axes[2].axis("off")
    plt.show()


    ipywidgets.interact(
        show_steps,
        n=ipywidgets.IntSlider(min=1, max=len(steps_sinogram), step=1, value=1),
        steps=ipywidgets.fixed(steps_sinogram),
        name=ipywidgets.fixed("Sinogram Steps")
    )

    ipywidgets.interact(
        show_steps,
        n=ipywidgets.IntSlider(min=1, max=len(steps_inversed), step=1, value=1),
        steps=ipywidgets.fixed(steps_inversed),
        name=ipywidgets.fixed("Reconstruction Steps")
    )

def main_UI():
    ipywidgets.interact(
        interactive_main,
        steps=ipywidgets.IntSlider(min=90, max=720, step=10, value=360, description="Steps"),
        detectors=ipywidgets.IntSlider(min=90, max=720, step=10, value=360, description="Detectors"),
        angle_of_cone=ipywidgets.IntSlider(min=10, max=180, step=5, value=90, description="Cone Angle"),
        filename=ipywidgets.Text(value='tomograf-obrazy/Kolo.jpg', description="Image Path")
    )


main_UI()
print(reconstructed_image is None)


interactive(children=(Text(value='tomograf-obrazy/Kolo.jpg', description='Image Path'), IntSlider(value=360, d…

False


In [50]:
output = ipywidgets.Output()
output_file_name = ipywidgets.Text(description="File name:", value="output")
patient_name = ipywidgets.Text(description="Name:", value="John Doe")
patient_id = ipywidgets.Text(description="ID:", value= "123456")
patient_birth = ipywidgets.DatePicker(description="Birth date:", value=datetime.date.today())
patient_sex = ipywidgets.Dropdown(description="Sex:", options=['M', 'F'], value='M')
date_of_exam = ipywidgets.DatePicker(description="Date of exam:", value=datetime.date.today())
comment = ipywidgets.Text(description="Comment:", value="Check-up")
save_button = ipywidgets.Button(description="Save as DICOM")
display(output_file_name, patient_name, patient_id, patient_birth, patient_birth, patient_sex, date_of_exam, comment)
display(save_button, output)

save_button.on_click(lambda b: save_dicom(b, output))

Text(value='output', description='File name:')

Text(value='John Doe', description='Name:')

Text(value='123456', description='ID:')

DatePicker(value=datetime.date(2025, 4, 16), description='Birth date:', step=1)

DatePicker(value=datetime.date(2025, 4, 16), description='Birth date:', step=1)

Dropdown(description='Sex:', options=('M', 'F'), value='M')

DatePicker(value=datetime.date(2025, 4, 16), description='Date of exam:', step=1)

Text(value='Check-up', description='Comment:')

Button(description='Save as DICOM', style=ButtonStyle())

Output()